In [ ]:
!pip install -U datasets

In [ ]:
# %% 1. การติดตั้งและ Import Library

# ติดตั้ง Library ที่จำเป็น
!pip install -q transformers[torch] torch accelerate scikit-learn

import pandas as pd
import numpy as np
import torch
import json
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.metrics import accuracy_score, f1_score
from tqdm.auto import tqdm

# ตรวจสอบและตั้งค่าอุปกรณ์ (GPU)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"อุปกรณ์ที่ใช้: {DEVICE}")


อุปกรณ์ที่ใช้: cuda


In [ ]:
# %% 2. การเตรียมข้อมูลและตัวอย่าง Few-Shot

print("กำลังโหลด Thai Fake News Detection Dataset...")
# โหลดชุดข้อมูลจาก Hugging Face Hub
raw_dataset = load_dataset("EXt1/Thai-True-Fake-News")

กำลังโหลด Thai Fake News Detection Dataset...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
from datasets import Dataset, DatasetDict

# แปลง Dataset เป็น DataFrame ก่อน
df = raw_dataset['train'].to_pandas()

# เปลี่ยนชื่อคอลัมน์ใน DataFrame
df = df.rename(columns={'Title': 'text', 'Verification_Status': 'label'})

# อัปเดต raw_dataset โดยแปลงจาก DataFrame กลับเป็น Dataset

# สร้าง Dataset ใหม่จาก DataFrame ที่เปลี่ยนชื่อคอลัมน์แล้ว
updated_dataset = Dataset.from_pandas(df)

# สร้าง DatasetDict ใหม่
raw_dataset = DatasetDict({
    'train': updated_dataset
})

print("เปลี่ยนชื่อคอลัมน์สำเร็จ:")
print(f"DataFrame columns: {df.columns.tolist()}")
print(f"Dataset features: {raw_dataset['train'].features}")
print("\nตัวอย่างข้อมูลหลังเปลี่ยนชื่อ:")
print(df[['text', 'label']].head())

เปลี่ยนชื่อคอลัมน์สำเร็จ:
DataFrame columns: ['Unnamed: 0', 'text', 'label']
Dataset features: {'Unnamed: 0': Value(dtype='float64', id=None), 'text': Value(dtype='string', id=None), 'label': Value(dtype='string', id=None)}

ตัวอย่างข้อมูลหลังเปลี่ยนชื่อ:
                                                text     label
0   กรมพัฒนาธุรกิจการค้าอนุญาตใบทะเบียนพาณิชย์ราย...  ข่าวปลอม
1   กรมการจัดหางานส่งเสริมให้ชาวไทยมีรายได้ เฉลี่...  ข่าวปลอม
2   ตลาดหลักทรัพย์แห่งประเทศไทยเปิดลงทุนเพื่อหาค่...  ข่าวปลอม
3   ตลาดหลักทรัพย์ฯ เปิดพอร์ตหุ้นธนาคารกำไรสูง 48...  ข่าวปลอม
4   ผู้บริหารธ. ออมสินไลน์เชิญชวนกู้เงินหลักหมื่น...  ข่าวปลอม


In [9]:

df = raw_dataset['train'].to_pandas()
df = df.dropna(subset=['text', 'label']).drop_duplicates(subset=['text'])

# จำลองคอลัมน์จากโจทย์
df['news_title'] = df['text'].str[:30]
df['news_content'] = df['text'].str[30:]

# --- คัดเลือกตัวอย่างคุณภาพสูงสำหรับ Few-Shot Prompt ---
# ตัวอย่างที่ 1: ข่าวจริง (Real News)
real_news_example = df[df['label'] == 'ข่าวจริง'].iloc[10] # เลือกตัวอย่างที่ชัดเจน
# ตัวอย่างที่ 2: ข่าวปลอม (Fake News)
fake_news_example = df[df['label'] == 'ข่าวปลอม'].iloc[5] # เลือกตัวอย่างที่ชัดเจน

# --- สร้างข้อมูลสำหรับ Test Set เพื่อจำลองการแข่งขัน ---
test_df = df.sample(n=10, random_state=42) # ลดขนาดเพื่อความรวดเร็วในการทดลอง
test_df['news_id'] = range(len(test_df))
test_df_labels = test_df['label'].apply(lambda x: 1 if x == 'fake' else 0).values

print("คัดเลือกตัวอย่าง Few-shot และเตรียม Test set สำเร็จ")
print("\n--- ตัวอย่างข่าวจริง ---")
print(f"Title: {real_news_example['news_title']}")
print("\n--- ตัวอย่างข่าวปลอม ---")
print(f"Title: {fake_news_example['news_title']}")


คัดเลือกตัวอย่าง Few-shot และเตรียม Test set สำเร็จ

--- ตัวอย่างข่าวจริง ---
Title: ปลัด พม. สั่งเร่งตรวจสอบข้อเท็

--- ตัวอย่างข่าวปลอม ---
Title:   เพิ่มเบี้ยผู้สูงอายุ คนละ 10


In [6]:
# %% 3. การโหลดโมเดล LLM และ Tokenizer

model_id = "Qwen/Qwen2-7B-Instruct"
print(f"กำลังโหลด Tokenizer และโมเดล LLM: '{model_id}'...")

tokenizer = AutoTokenizer.from_pretrained(model_id)
model_llm = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16, # ใช้ bfloat16 เพื่อลดการใช้ VRAM และเพิ่มความเร็ว
    device_map="auto"           # โหลดโมเดลข้าม GPU อัตโนมัติหากมีหลายตัว
)
model_llm.eval() # ตั้งเป็น evaluation mode

print("โหลดโมเดล LLM สำเร็จ")


กำลังโหลด Tokenizer และโมเดล LLM: 'Qwen/Qwen2-7B-Instruct'...


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

โหลดโมเดล LLM สำเร็จ


In [10]:
# %% 4. การออกแบบ Prompt (Few-Shot & Chain-of-Thought)

def create_few_shot_prompt(news_title, news_content):
    """
    สร้าง Prompt ที่สมบูรณ์แบบ Few-shot และ Chain-of-Thought
    """
    # นี่คือ Chat Template ที่เราจะส่งให้โมเดล
    chat = [
        # 1. System Message: กำหนดบทบาทและคำสั่งหลัก
        {"role": "system", "content": "คุณคือนักสืบสวนข่าวปลอมผู้เชี่ยวชาญด้านการเงิน คุณต้องวิเคราะห์ข่าวตามขั้นตอนที่กำหนด และตอบกลับเป็น JSON object ที่มี key 'analysis' และ 'prediction' เท่านั้น โดยค่าของ prediction ต้องเป็น 'เชื่อถือได้' หรือ 'ข้อมูลบิดเบือน' อย่างใดอย่างหนึ่ง"},

        # 2. Few-shot Example 1 (ข่าวจริง)
        {"role": "user", "content": f"""จงวิเคราะห์ข่าวต่อไปนี้:
หัวข้อ: "{real_news_example['news_title']}"
เนื้อหา: "{real_news_example['news_content']}"

ขั้นตอนการวิเคราะห์:
1. มีการอ้างแหล่งข้อมูลที่ไม่น่าเชื่อถือหรือไม่?
2. มีการใช้ภาษาที่ชี้นำอารมณ์มากกว่าข้อเท็จจริงหรือไม่?
3. มีการกล่าวอ้างเกินจริง (เช่น "การันตีผลตอบแทน") หรือไม่?
4. สรุป: ข่าวนี้มีแนวโน้มเป็นข้อมูลบิดเบือนหรือไม่? ให้ตอบว่า "ข่าวจริง" หรือ "ข่าวปลอม"
"""},
        {"role": "assistant", "content": """{
    "analysis": "1. ข่าวมีการอ้างอิงถึงหน่วยงานภาครัฐและข้อมูลที่ตรวจสอบได้ 2. ภาษาที่ใช้เป็นทางการและเน้นข้อเท็จจริง ไม่มีการชี้นำอารมณ์ 3. ไม่มีการกล่าวอ้างเกินจริงหรือรับประกันผลตอบแทน 4. สรุปแล้ว ข่าวนี้มีความน่าเชื่อถือสูง",
    "prediction": "ข่าวจริง"
}"""},

        # 3. Few-shot Example 2 (ข่าวปลอม)
        {"role": "user", "content": f"""จงวิเคราะห์ข่าวต่อไปนี้:
หัวข้อ: "{fake_news_example['news_title']}"
เนื้อหา: "{fake_news_example['news_content']}"

ขั้นตอนการวิเคราะห์:
1. มีการอ้างแหล่งข้อมูลที่ไม่น่าเชื่อถือหรือไม่?
2. มีการใช้ภาษาที่ชี้นำอารมณ์มากกว่าข้อเท็จจริงหรือไม่?
3. มีการกล่าวอ้างเกินจริง (เช่น "การันตีผลตอบแทน") หรือไม่?
4. สรุป: ข่าวนี้มีแนวโน้มเป็นข้อมูลบิดเบือนหรือไม่? ให้ตอบว่า "ข่าวจริง" หรือ "ข่าวปลอม"
"""},
        {"role": "assistant", "content": """{
    "analysis": "1. ข่าวไม่มีการอ้างอิงแหล่งข้อมูลที่ชัดเจน และมาจากบุคคลที่ไม่เป็นที่รู้จัก 2. ภาษาที่ใช้เร่งเร้าและกระตุ้นความโลภ ('ด่วน', 'รวยเร็ว') มากกว่าให้ข้อมูล 3. มีการกล่าวอ้างถึงผลตอบแทนที่สูงเกินจริงและง่ายดาย 4. สรุปแล้ว ข่าวนี้มีลักษณะของข้อมูลบิดเบือนเพื่อหลอกลวง",
    "prediction": "ข่าวปลอม"
}"""},

        # 4. User Query (ข่าวที่เราต้องการวิเคราะห์)
        {"role": "user", "content": f"""จงวิเคราะห์ข่าวต่อไปนี้:
หัวข้อ: "{news_title}"
เนื้อหา: "{news_content}"

ขั้นตอนการวิเคราะห์:
1. มีการอ้างแหล่งข้อมูลที่ไม่น่าเชื่อถือหรือไม่?
2. มีการใช้ภาษาที่ชี้นำอารมณ์มากกว่าข้อเท็จจริงหรือไม่?
3. มีการกล่าวอ้างเกินจริง (เช่น "การันตีผลตอบแทน") หรือไม่?
4. สรุป: ข่าวนี้มีแนวโน้มเป็นข้อมูลบิดเบือนหรือไม่? ให้ตอบว่า "ข่าวจริง" หรือ "ข่าวปลอม"
"""},
    ]
    return chat

# ทดสอบสร้าง Prompt
test_prompt = create_few_shot_prompt("ทดสอบหัวข้อ", "ทดสอบเนื้อหา")
print("สร้าง Template Prompt สำเร็จ")
# print(json.dumps(test_prompt, indent=2, ensure_ascii=False)) # สามารถ uncomment เพื่อดูโครงสร้างเต็มๆ ได้


สร้าง Template Prompt สำเร็จ


In [11]:
# %% 5. การทำนายผลด้วย LLM และการ Parsing

results = []
for _, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Predicting with LLM"):
    # สร้าง Prompt สำหรับข่าวแต่ละชิ้น
    prompt = create_few_shot_prompt(row['news_title'], row['news_content'])

    # แปลง Prompt เป็น Token IDs และส่งเข้า GPU
    inputs = tokenizer.apply_chat_template(
        prompt, tokenize=True, add_generation_prompt=True, return_tensors="pt"
    ).to(DEVICE)

    # จำกัดความยาว Input เพื่อป้องกัน Error
    if inputs.shape[1] > 4096:
        inputs = inputs[:, -4096:]

    # สั่งให้โมเดลสร้างข้อความ
    with torch.no_grad():
        outputs = model_llm.generate(
            inputs,
            max_new_tokens=256,         # จำกัดความยาวของคำตอบ
            do_sample=False,            # ไม่ต้องสุ่มคำตอบเพื่อให้ผลลัพธ์คงที่
            pad_token_id=tokenizer.eos_token_id
        )

    # ถอดรหัสคำตอบ
    response_text = tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True)

    # พยายาม Parse JSON จากคำตอบ
    try:
        start_index = response_text.find('{')
        end_index = response_text.rfind('}')
        json_part = response_text[start_index : end_index + 1]
        data = json.loads(json_part)
        prediction = data.get('prediction', 'error')
    except (json.JSONDecodeError, AttributeError, ValueError):
        prediction = 'error' # หากเกิดข้อผิดพลาดในการ parse

    results.append(prediction)

# เพิ่มผลลัพธ์ลงใน DataFrame
test_df['llm_prediction'] = results

print("\nทำนายผลด้วย LLM สำเร็จ")
print("ตัวอย่างผลการทำนาย:")
print(test_df[['news_title', 'label', 'llm_prediction']].head(10))


Predicting with LLM:   0%|          | 0/10 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p'


ทำนายผลด้วย LLM สำเร็จ
ตัวอย่างผลการทำนาย:
                          news_title     label llm_prediction
4283  สธ. ร่วมมือกับสปสช. เพิ่มชุดตร  ข่าวจริง       ข่าวจริง
5495  กทพ. แจ้งปิดและเบี่ยงการจราจรท  ข่าวจริง       ข่าวจริง
2357   ออมสินเปิดให้กู้ 500,000 บาท   ข่าวปลอม       ข่าวจริง
4460  ราชกิจจาประกาศ เคอร์ฟิวทั่วราช  ข่าวจริง       ข่าวจริง
2908   คลิปเสียงจากไปรษณีย์ แจ้งลูกค  ข่าวปลอม       ข่าวจริง
3515  คนที่มีอาการหวัดร้อนไม่แนะนำทา  ข่าวจริง       ข่าวปลอม
1201   กินฟ้าทะลายโจร 3 แคปซูล ก่อนอ  ข่าวปลอม       ข่าวปลอม
3497  ปัญหาด้านการมองเห็น อาจเสี่ยงก  ข่าวจริง       ข่าวปลอม
5588  อุทยานฯ เขาคิชฌกูฏ เปิดนมัสการ  ข่าวจริง       ข่าวจริง
1975   ไปรษณีย์ไทย ใช้เบอร์ 081-3607  ข่าวปลอม       ข่าวจริง


In [14]:
test_df

,Unnamed: 0,text,label,news_title,news_content,news_id,llm_prediction,predicted_label
4283,2570.0,สธ. ร่วมมือกับสปสช. เพิ่มชุดตรวจ HIV ด้วยตนเอง...,ข่าวจริง,สธ. ร่วมมือกับสปสช. เพิ่มชุดตร,วจ HIV ด้วยตนเอง – ไวรัสตับอักเสบบีและซี ฟรีใน...,0,ข่าวจริง,-1
5495,NaN,กทพ. แจ้งปิดและเบี่ยงการจราจรทางพิเศษกาญจนาภิเ...,ข่าวจริง,กทพ. แจ้งปิดและเบี่ยงการจราจรท,างพิเศษกาญจนาภิเษก (บางพลี-สุขสวัสดิ์) เพื่อปร...,1,ข่าวจริง,-1
2357,333.0,"ออมสินเปิดให้กู้ 500,000 บาท ผ่อนนาน 2 ปี ผ่า...",ข่าวปลอม,"ออมสินเปิดให้กู้ 500,000 บาท",ผ่อนนาน 2 ปี ผ่านเพจ ธนาคาร ออมสิน,2,ข่าวจริง,-1
4460,5766.0,ราชกิจจาประกาศ เคอร์ฟิวทั่วราชอาณาจักร งดเดินท...,ข่าวจริง,ราชกิจจาประกาศ เคอร์ฟิวทั่วราช,อาณาจักร งดเดินทางข้ามจังหวัด,3,ข่าวจริง,-1
2908,4696.0,คลิปเสียงจากไปรษณีย์ แจ้งลูกค้าให้ระวังพัสดุม...,ข่าวปลอม,คลิปเสียงจากไปรษณีย์ แจ้งลูกค,้าให้ระวังพัสดุมีเชื้อโควิด-19,4,ข่าวจริง,-1
3515,9168.0,คนที่มีอาการหวัดร้อนไม่แนะนำทานขิง เพราะไม่ได้...,ข่าวจริง,คนที่มีอาการหวัดร้อนไม่แนะนำทา,นขิง เพราะไม่ได้ช่วยให้หายและอาจทำให้อาการทรุดลง,5,ข่าวปลอม,-1
1201,6991.0,กินฟ้าทะลายโจร 3 แคปซูล ก่อนออกจากบ้าน สามารถ...,ข่าวปลอม,กินฟ้าทะลายโจร 3 แคปซูล ก่อนอ,อกจากบ้าน สามารถป้องกันเชื้อโควิด 19 ได้ 12 ชม.,6,ข่าวปลอม,-1
3497,4144.0,ปัญหาด้านการมองเห็น อาจเสี่ยงก่อให้เกิดภาวะสมอ...,ข่าวจริง,ปัญหาด้านการมองเห็น อาจเสี่ยงก,่อให้เกิดภาวะสมองเสื่อม ?,7,ข่าวปลอม,-1
5588,8475.0,อุทยานฯ เขาคิชฌกูฏ เปิดนมัสการรอยพระพุทธบาทเขา...,ข่าวจริง,อุทยานฯ เขาคิชฌกูฏ เปิดนมัสการ,รอยพระพุทธบาทเขาคิชฌกูฏ ประจำปี 2567,8,ข่าวจริง,-1
1975,4079.0,ไปรษณีย์ไทย ใช้เบอร์ 081-3607134 แจ้งมีพัสดุต...,ข่าวปลอม,ไปรษณีย์ไทย ใช้เบอร์ 081-3607,134 แจ้งมีพัสดุตกค้าง,9,ข่าวจริง,-1


In [17]:
# %% 6. การประเมินผลและสร้างไฟล์ Submission

# แปลง prediction ที่เป็น string เป็น label ตัวเลข (0 หรือ 1)
def to_label(pred_text):
    if "ข่าวปลอม" in pred_text:
        return 1
    elif "ข่าวจริง" in pred_text:
        return 0
    else:
        return -1 # คืนค่า -1 หากเป็น Error หรือคำตอบไม่ตรงรูปแบบ

test_df['predicted_label'] = test_df['llm_prediction'].apply(to_label)

# กรองแถวที่เกิด Error ออกก่อนประเมินผล
eval_df = test_df[test_df['predicted_label'] != -1]
actual_labels = eval_df['label'].apply(lambda x: 1 if x == 'fake' else 0)
predicted_labels = eval_df['predicted_label']


# ประเมินผลเทียบกับคำตอบจริง (เพื่อดูประสิทธิภาพของ Prompt)
if not eval_df.empty:
    accuracy = accuracy_score(actual_labels, predicted_labels)
    f1 = f1_score(actual_labels, predicted_labels)
    print("\n--- ประสิทธิภาพของโมเดลบน Test Set ---")
    print(f"accuracy: {accuracy:.4f}")
    print(f"F1-Score: {f1:.4f}")
    print(f"จำนวนที่ทำนายได้สำเร็จ: {len(eval_df)}/{len(test_df)}")
else:
    print("\nไม่สามารถประเมินผลได้เนื่องจาก LLM ไม่ได้ให้คำตอบที่ถูกต้อง")

# --- สร้างไฟล์ Submission ---
# สำหรับแถวที่เกิด Error อาจจะต้องใช้วิธีสำรอง หรือเติมค่าที่พบบ่อยที่สุด (Majority class)
# ในที่นี้ เราจะเติม 0 สำหรับแถวที่เกิด error
final_predictions = test_df['predicted_label'].copy()
final_predictions[final_predictions == -1] = 0 # เติม 0 (เชื่อถือได้) สำหรับแถวที่ error

submission_df = pd.DataFrame({
    'news_id': test_df['news_id'],
    'label': final_predictions.astype(int)
})

print("\nตัวอย่างข้อมูลในไฟล์ Submission:")
print(submission_df.head())

# บันทึกเป็นไฟล์ CSV
submission_df.to_csv("submission_llm.csv", index=False)

print("\nสร้างไฟล์ submission_llm.csv สำเร็จ!")



--- ประสิทธิภาพของโมเดลบน Test Set ---
accuracy: 0.7000
F1-Score: 0.0000
จำนวนที่ทำนายได้สำเร็จ: 10/10

ตัวอย่างข้อมูลในไฟล์ Submission:
      news_id  label
4283        0      0
5495        1      0
2357        2      0
4460        3      0
2908        4      0

สร้างไฟล์ submission_llm.csv สำเร็จ!


In [18]:
final_predictions

,predicted_label
4283,0
5495,0
2357,0
4460,0
2908,0
3515,1
1201,1
3497,1
5588,0
1975,0


In [19]:
submission_df

,news_id,label
4283,0,0
5495,1,0
2357,2,0
4460,3,0
2908,4,0
3515,5,1
1201,6,1
3497,7,1
5588,8,0
1975,9,0
